# Bibliotheques

In [4]:
import pandas as pd
from prophet import Prophet
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
from catboost import CatBoostRegressor
from datetime import datetime
from pmdarima.arima import auto_arima
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.io as pio
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, TimeSeriesSplit
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
import dtale


# Chargement des données

In [16]:
df = pd.read_excel("C:\\Users\WINDOWS\\Desktop\\Nouveau dossier\\Nouveau dossier\\ressources_memoire\\P_BB.xlsx")
df

,date,Canal,Region,Sous_famille,ventes
0,2019-01-01,C1,A,P1,80.725952
1,2019-01-01,C1,A,P2,46.322000
2,2019-01-01,C1,A,P3,41.165952
3,2019-01-01,C1,B,P1,36.760256
4,2019-01-01,C1,B,P2,37.618800
...,...,...,...,...,...
1615,2023-12-01,C3,B,P2,167.108800
1616,2023-12-01,C3,B,P3,25.097600
1617,2023-12-01,C3,C,P1,11.124668
1618,2023-12-01,C3,C,P2,222.511200


## Visualisation

In [17]:


# Parcourir les sous-familles
for sous_famille in df['Sous_famille'].unique():
    # Sélectionner les données de la sous-famille en cours
    ventes_sous_famille = df[df['Sous_famille'] == sous_famille]

    # Créer une nouvelle figure avec Plotly
    fig = go.Figure()

    # Parcourir les régions et les canaux pour ajouter des traces
    for region in ventes_sous_famille['Region'].unique():
        ventes_region = ventes_sous_famille[ventes_sous_famille['Region'] == region]
        for canal in ventes_region['Canal'].unique():
            ventes_canaux = ventes_region[ventes_region['Canal'] == canal]
            fig.add_trace(go.Scatter(
                x=ventes_canaux['date'], 
                y=ventes_canaux['ventes'], 
                mode='lines', 
                name=f'{region} - {canal}',
                visible=True  # Initial state
            ))

    # Créer des boutons pour la liste déroulante
    buttons = []
    for trace in fig.data:
        button = dict(
            method='update',
            label=trace.name,
            args=[{'visible': [t == trace for t in fig.data]},
                  {'title': f'Ventes de {sous_famille} - {trace.name}'}]
        )
        buttons.append(button)

    # Ajouter les boutons à la mise en page
    fig.update_layout(
        updatemenus=[dict(
            active=0,
            buttons=buttons,
            x=1.15,
            xanchor='left',
            y=1.15,
            yanchor='top'
        )]
    )

    # Ajouter une légende, un titre et des étiquettes d'axes
    fig.update_layout(
        title=f'Ventes de {sous_famille}',
        xaxis_title='Date',
        yaxis_title='Ventes',
        xaxis=dict(tickangle=45),
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=0.75),
        margin=dict(l=0, r=0, t=50, b=0),
    )

    # Afficher la figure
    fig.show()


# AED

In [18]:
stats_desc = df.describe()
# Affichage des statistiques descriptives du dataset
print(stats_desc)

                      date       ventes
count                 1620  1620.000000
mean   2021-06-16 06:24:00   117.277931
min    2019-01-01 00:00:00     0.516672
25%    2020-03-24 06:00:00    33.364725
50%    2021-06-16 00:00:00    83.062031
75%    2022-09-08 12:00:00   161.101062
max    2023-12-01 00:00:00  1391.468691
std                    NaN   125.652791


## Missing values

In [19]:
# Vérification des valeurs manquantes dans le dataset
missing_values = df.isnull().sum()

# Affichage des valeurs manquantes
print(missing_values)

# Vérification si toutes les valeurs manquantes sont à zéro
if missing_values.sum() == 0:
    print("Les données sont propres")
else:
    print("Il y a des valeurs manquantes dans les données")


date            0
Canal           0
Region          0
Sous_famille    0
ventes          0
dtype: int64
Les données sont propres


## date to datetime

In [20]:
df['date']=pd.to_datetime(df['date'])

# visualisation 


## Par Canal

In [21]:


# Grouper les données par 'Canal' et 'Sous_famille' et sommer les ventes
sales_by_canal_sous_famille = df.groupby(['Canal', 'Sous_famille'])['ventes'].sum().reset_index()

# Obtenir les canaux uniques
canals = df['Canal'].unique()

# Définir une séquence de couleurs personnalisée
couleurs_personnalisees = ['#BDB498', '#0E2C54', '#CAE8FF']

# Créer les sous-graphiques pour chaque canal
fig = make_subplots(rows=1, cols=len(canals), subplot_titles=[f'Canal {canal}' for canal in canals], specs=[[{'type':'pie'}] * len(canals)])

for i, canal in enumerate(canals):
    # Filtrer les données pour le canal spécifique
    data = sales_by_canal_sous_famille[sales_by_canal_sous_famille['Canal'] == canal]
    
    # Ajouter un diagramme circulaire au sous-graphe
    fig.add_trace(go.Pie(
        labels=data['Sous_famille'],
        values=data['ventes'],
        name=f'Canal {canal}',
        hole=0.3,
        marker=dict(colors=couleurs_personnalisees)
    ), row=1, col=i+1)

# Mettre à jour la disposition du graphe pour utiliser le thème 'plotly_dark'
fig.update_layout(
    title_text='Distribution des Ventes par Sous-famille et Canal',
    template='plotly_dark'
)

fig.show()


## par region

In [22]:

# Grouper les données par 'Region' et 'Sous_famille' et sommer les ventes
sales_by_region_sous_famille = df.groupby(['Region', 'Sous_famille'])['ventes'].sum().reset_index()

# Obtenir les régions uniques
regions = df['Region'].unique()

# Définir une séquence de couleurs personnalisée
couleurs_personnalisees = ['#BDB498', '#0E2C54', '#CAE8FF']

# Créer les sous-graphiques pour chaque région
fig = make_subplots(rows=1, cols=len(regions), subplot_titles=[f'Region {region}' for region in regions], specs=[[{'type':'pie'}] * len(regions)])

for i, region in enumerate(regions):
    # Filtrer les données pour la région spécifique
    data = sales_by_region_sous_famille[sales_by_region_sous_famille['Region'] == region]
    
    # Ajouter un diagramme circulaire au sous-graphe
    fig.add_trace(go.Pie(
        labels=data['Sous_famille'],
        values=data['ventes'],
        name=f'Region {region}',
        hole=0.3,
        marker=dict(colors=couleurs_personnalisees)
    ), row=1, col=i+1)

# Mettre à jour la disposition du graphe pour utiliser le thème 'plotly_dark'
fig.update_layout(
    title_text='Distribution des Ventes par Sous-famille et Région',
    template='plotly_dark'
)

fig.show()
